# About this script
This script is written in python 2.7 using the jupyter notebook app. It relies on externally linked libraries that are freely available and easy to install for Linux systems, but a bitch to get working on windows. The necessary prerequisits to use this program are: <br><br>
1) FigureGenerator49.f90 - the uncompiled fortran code is available at www.caseydietrich.com <br>
2) a fortran compiler - (gfortran works well) to compile the previous script into a binary executable <br>
3) GMT 4.5.x - This is available in repositories of most debian based Linux OSs and is easily installed using apt-get.<br>
4) ghostscript - also available from repositories using apt-get<br>
5) imagemagick - also available from repositories using apt-get<br><br>
Once you have a system up and running with these prerequisits you should get example input files for FigureGenerator from Casey Dietrich's website and edit them to fit your needs. 
!Hint! Keep your temporary folder path as short as possible to avoid getting overflow errors from Fortran.<br><br>
In a test case the entire thing took about 7 minutes to run through two storm folders, so a harddrive with ~20 storms should take around 1 hour and 10 minutes.<br>
Test case 2: 25 storm folder in 1 hour and 46 minutes.

# Import modules

In [1]:
import os
import subprocess
import shutil

# Define sub-functions

In [2]:
def plot_hydrograph(input_file):
    #Import modules
    %matplotlib inline
    import matplotlib as mpl
    mpl.style.use(['ggplot','bmh'])
    import matplotlib.pyplot as plt
    import pandas as pd
    import os
    import numpy as np

    #List of Stations 
    stations = ['NOAA - St. Petersburg',
                'NOAA - Port Manatee',
                "USGS 02299735 - Venice Inlet at Crow's Nest Marina",
                'USGS 02293332 - Charlotte Harbor at Port Boca Grande',
                'USGS 02299230 - Myakka River at North Port Charlotte',
                'USGS 02297460 - Peace River at Harbour Heights',
                'NOAA - Fort Myers',
                'USF - Matanzas Pass',
                'USF - Big Carlos Pass',
                'USF - Spring Creek',
                'NOAA - Naples',
                'USGS 02290928 - Barron River at Everglades City'
               ]

    #Import Data
    f = open(input_file,'r')

    header     = f.readline()
    Header     = " ".join(header.split())
    parameters = f.readline()
    Parameters = " ".join(parameters.split()).split(' ',5)

    NTRSPE            = int(Parameters[0])
    NSTAE             = int(Parameters[1])
    DT_TIMES_NSPOOLGE = Parameters[2]
    NSPOOLGE          = Parameters[3]
    IRTYPE            = Parameters[4]

    index    = np.arange(NTRSPE)
    Index    = map(str,index.tolist())
    columns  = ['TIME','IT']
    numbered = np.arange(NSTAE)+1
    Numbers  = map(str,numbered.tolist())
    columns.extend(Numbers)

    df = pd.DataFrame(index=index,columns=columns)

    for i in range(NTRSPE):
        data = f.readline()
        Data = map(float," ".join(data.split()).split(' ',1))
        df['TIME'][i] = Data[0]/(24*3600) 
        df['IT'][i]   = Data[1]
        for j in range(NSTAE):
            data2 = f.readline()
            Data2 = map(float," ".join(data2.split()).split(' ',1))
            if Data2[1] == -99999:
                df[str(j+1)][i] = np.nan
            else:
                df[str(j+1)][i] = Data2[1]
    f.close()

    #Plot Hydrographs
    fig, axs = plt.subplots(NSTAE,1,figsize=(12,NSTAE*3)) 
    fig.suptitle(input_file.rsplit('/',2)[1],fontsize=18)
    for i in range(NSTAE):
        axs[i].grid(True)
        axs[i].set_title(stations[i])
        axs[i].plot(df['TIME'].tolist(),
                    df[str(i+1)].tolist())
        axs[i].set_ylabel("Elevation (m)")
        axs[i].set_xlabel("Time (days)")        
        axs[i].tick_params(axis='both',labelsize=12)   
    for i in range(NSTAE-1):
        axs[i].set_xlabel("")
        axs[i].tick_params(labelbottom='off') # labels along the bottom edge are off
    fig.savefig(input_file.rsplit('/',1)[0]+'/hydrographs.jpg',dpi = 200)
    fig.savefig(input_file.rsplit('/',1)[0]+'/hydrographs.pdf',dpi = 100)
    plt.close(fig)

In [3]:
def make_figures(parent_directory,Sub_Directs,figgen,Input_Files,other_files):
    
    os.chdir(parent_directory)

    for root, dirs, files in os.walk(parent_directory):
        if root != parent_directory and root.rsplit("/",1)[1] not in Sub_Directs:
            os.chdir(root)
            print(root)
            #Hydrograph
            if os.path.exists(root+"/fort.61") == True:
                print("Plotting hydrograph "+root+"/fort.61")
                plot_hydrograph(root+"/fort.61")
            #FigGen49
            if os.path.exists(root+"/DSPRatmaxele.63") == False:
                sub_directs = Sub_Directs[:-1]
                input_files = Input_Files[:-1]
            else:
                sub_directs = Sub_Directs
                input_files = Input_Files
                
            for other_file in other_files:
                if os.path.exists(root+"/"+other_file)==False:
                    shutil.copy(parent_directory+"/"+other_file,root+"/"+other_file)
            for input_file in input_files:
                if os.path.exists(root+"/"+input_file)==False:
                    shutil.copy(parent_directory+"/"+input_file,root+"/"+input_file)
            for directory in sub_directs:
                if os.path.exists(root+"/"+directory) == False:
                    os.mkdir(directory)
            for input_file in input_files[0:-1]:
                subprocess.Popen([root+"/"+figgen,"-I ",input_file])
            #NOTE - 
            #If this subprocess throws an error due to issues with the .inp file it will cause the script to
            #begin processing all folders at once, most likely overwhelming your system and causing your computer to 
            #freeze up. This is a bug that needs to be fixed! JD-2015/12/08
            #Parallelization in this script is pretty basic and should be improved
            process1 = subprocess.Popen([root+"/"+figgen,"-I",input_files[-1]],stdout=subprocess.PIPE)
            process1.wait()

In [4]:
def delete_files(parent_directory,sub_directs,figgen,input_files,other_files):

    os.chdir(parent_directory)
    if os.path.exists(parent_directory+"QC/") == False:
        os.mkdir(parent_directory+"QC/")
    for root, dirs, files in os.walk(parent_directory):
        bundle_folder = root.rsplit("/",1)[1]
        if root != parent_directory and bundle_folder not in sub_directs and root[-2:]!="QC":
            os.chdir(root)
            print(root)
            qcfolder = parent_directory+"QC/"+bundle_folder+"_QC"
            if os.path.exists(qcfolder) == False:
                os.mkdir(qcfolder)
            for filed in files:
                if filed[-3:] == ".ps":
                    os.remove(root+"/"+filed)
            for other_file in other_files:
                if os.path.exists(root+"/"+other_file)== True:
                    os.remove(root+"/"+other_file)
            for input_file in input_files:
                if os.path.exists(root+"/"+input_file)== True:
                    os.remove(root+"/"+input_file)
            for directory in sub_directs:
                if os.path.exists(root+"/"+directory) == True:
                    os.rmdir(directory)
            for filed in files:
                if filed[-4:] == ".jpg" or filed[-4:] == ".pdf":
                    shutil.move(filed,qcfolder)

# Set variables

In [9]:
parent_directory = "/mnt/runsv7/trop/"
figgen = "FigureGen49.sh"
Sub_Directs = ["T1",
               "T2",
               "T3",
               "T4",
               "T5",
               "T6",
               "T7",
               "T8",
               "T9"
              ]
Input_Files = ["maxele_overview.inp",
               "maxvel_overview.inp",
               "maxwvel_overview.inp",
               "minpr_overview.inp",
               "maxrs_overview.inp",                   
               "HSatmaxele_overview.inp",                   
               "TPSatmaxele_overview.inp",                   
               "DIRatmaxele_overview.inp",
               "DSPRatmaxele_overview.inp",
              ]
other_files = ["RAMPP_Logo.eps",
               "Default2.pal",
               "labels_SWFL.txt",
               figgen
              ]

# Make Figures

In [10]:
%time make_figures(parent_directory,Sub_Directs,figgen,Input_Files,other_files)

/mnt/runsv7/trop/OS1_HC_LF_D_0001_008
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_D_0001_008/fort.61
/mnt/runsv7/trop/.ipynb_checkpoints
/mnt/runsv7/trop/OS1_HC_LF_B_0005_005
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_B_0005_005/fort.61
/mnt/runsv7/trop/OS1_HC_LF_C_0001_007
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_C_0001_007/fort.61
/mnt/runsv7/trop/OS1_HC_LF_C_0001_008
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_C_0001_008/fort.61
/mnt/runsv7/trop/OS1_HC_LF_C_0005_005
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_C_0005_005/fort.61
/mnt/runsv7/trop/OS1_HC_LF_C_0005_006
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_C_0005_006/fort.61
/mnt/runsv7/trop/OS1_HC_LF_D_0001_005
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_D_0001_005/fort.61
/mnt/runsv7/trop/OS1_HC_LF_D_0001_006
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_D_0001_006/fort.61
/mnt/runsv7/trop/OS1_HC_LF_D_0001_007
Plotting hydrograph /mnt/runsv7/trop/OS1_HC_LF_D_0001_007/fort.61
/mnt/runsv7/trop/OS1_HC_LF_D

# Delete unnecessary files

In [11]:
%time delete_files(parent_directory,Sub_Directs,figgen,Input_Files,other_files)

/mnt/runsv7/trop/OS1_HC_LF_D_0001_008
/mnt/runsv7/trop/.ipynb_checkpoints
/mnt/runsv7/trop/OS1_HC_LF_B_0005_005
/mnt/runsv7/trop/OS1_HC_LF_C_0001_007
/mnt/runsv7/trop/OS1_HC_LF_C_0001_008
/mnt/runsv7/trop/OS1_HC_LF_C_0005_005
/mnt/runsv7/trop/OS1_HC_LF_C_0005_006
/mnt/runsv7/trop/OS1_HC_LF_D_0001_005
/mnt/runsv7/trop/OS1_HC_LF_D_0001_006
/mnt/runsv7/trop/OS1_HC_LF_D_0001_007
/mnt/runsv7/trop/OS1_HC_LF_D_0004_005
/mnt/runsv7/trop/OS1_HC_LF_D_0004_006
/mnt/runsv7/trop/OS1_HC_LF_D_0007_006
/mnt/runsv7/trop/OS1_HC_LF_D_0007_007
/mnt/runsv7/trop/OS1_HC_LF_D_0008_009
/mnt/runsv7/trop/OS1_HC_LF_D_0009_005
/mnt/runsv7/trop/OS1_HC_LF_D_0009_006
/mnt/runsv7/trop/OS1_HC_LF_D_0009_007
/mnt/runsv7/trop/OS1_HC_LF_D_0014_005
/mnt/runsv7/trop/OS1_HC_LF_D_0015_005
/mnt/runsv7/trop/OS1_TS_LF_C_0007_005
/mnt/runsv7/trop/OS1_TS_LF_D_0003_006
/mnt/runsv7/trop/OS1_TS_LF_D_0003_007
CPU times: user 80 ms, sys: 144 ms, total: 224 ms
Wall time: 1.48 s


Created by: Jared Dorvinen on November 23rd 2015 <br>
contact at: jdorvinen@dewberry.com